In [2]:
import cv2
import glob
import os
from xlrd import open_workbook
import numpy as np
import pickle

DATA_DIR = '/home/ubuntu/mtcnn_aligned_images/'
LABELS_FILE = '/home/ubuntu/ECUST_FBP/scores/private_generic_all.xlsx'


def create_dataset(target_size):
    X = []
    y = []
    labels_dict = get_labels_dict()
   # img_files = glob.glob(DATA_DIR+'*.jpg') # glob library to get all files in path as list
    #print(f'reading {len(img_files)} images into dataset')
    with open('average_scores.pkl', 'wb') as f:
        pickle.dump(labels_dict, f, pickle.HIGHEST_PROTOCOL)
        
    for root, dirs, files in os.walk(DATA_DIR):
     for file in files: 
        
        f = os.path.join(root, file)
         # open-cv library to read and transform images
        if file in labels_dict:
           img = preprocess_image(cv2.imread(f), target_size)
           X.append(img)
           y.append(labels_dict[os.path.split(f)[-1]])   
    return np.array(X), np.array(y)

def get_labels_dict():
  
    wb=open_workbook(LABELS_FILE)
    ws = wb.sheet_by_index(0)
    
    labels_dict = {}

    for i in range(ws.nrows-1):
       #print(ws.cell(i+1,0).value) 
       img = str(ws.cell(i+1,1).value)+'.jpg'
       label = ws.cell(i+1,2).value
       labels_dict[img] = float(label)
    print(len(labels_dict))    
    return labels_dict

def preprocess_image(image,target_size):
    return cv2.resize(cv2.cvtColor(image, cv2.COLOR_BGR2RGB),target_size) / .255

target_size = (224,224)
X,y = create_dataset(target_size)
print(X.shape)


1175
(1115, 224, 224, 3)


In [4]:
with open('average_scores.pkl', 'rb') as f:
        scores = pickle.load(f)

In [5]:
scores

{'112.jpg': 5.355855855855856,
 '122.jpg': 6.380787037037036,
 '130.jpg': 7.134146341463414,
 '14.jpg': 5.786706349206349,
 '15.jpg': 6.296296296296297,
 '42.jpg': 7.809523809523809,
 '56.jpg': 7.35,
 'andreea-pop-bTVliFTsdHI-unsplash.jpg': 5.715873015873016,
 'andriyko-podilnyk-KyQxMKaPTM0-unsplash.jpg': 6.566666666666666,
 'anthony-tran-6s3L8El33QE-unsplash.jpg': 7.5,
 'ashkan-forouzani-rHuReQyQ72c-unsplash.jpg': 6.166666666666667,
 'asian-2302727_1920.jpg': 7.238095238095238,
 'austin-distel-7bMdiIqz_J4-unsplash.jpg': 7.052325581395348,
 'beard-1845166_1920.jpg': 6.876984126984128,
 'beautiful-1867431_1920.jpg': 5.5,
 'beauty-4175607_1920.jpg': 6.368421052631579,
 'black-professional-4334648_1920.jpg': 5.386363636363637,
 'court-cook-TSZo17r3m0s-unsplash.jpg': 6.523809523809524,
 'daniel-tong-Vaznn9k5GqQ-unsplash.jpg': 7.833333333333333,
 'darius-bashar-8Kfyz3FMjtE-unsplash.jpg': 5.929824561403509,
 'david-todd-mccarty-NXlQ2PnzMGg-unsplash.jpg': 4.63963963963964,
 'dima-dallacqua-Nz

In [8]:
#MobileNetV2

import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import multi_gpu_model
from tensorflow.keras.models import load_model
import tensorflow.keras.backend as K

basemodel = MobileNetV2(include_top=False, pooling='avg', weights='imagenet')


X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.30, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=1/3, random_state=42)

from tensorflow.keras.preprocessing.image import ImageDataGenerator
test_val_datagen = ImageDataGenerator()
train_datagen = ImageDataGenerator(horizontal_flip=True,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2)

batch_size = 32
train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)
val_generator = test_val_datagen.flow(X_val, y_val, batch_size=batch_size)
test_generator = test_val_datagen.flow(X_test, y_test, batch_size=batch_size, shuffle=False)

model = Sequential()
model.add(basemodel)
model.add(Dense(1))
epochs = 10
lr=0.001
model.layers[0].trainable = False
model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=lr))
history1 = model.fit(
    train_generator,
    epochs=epochs,
    steps_per_epoch=len(y_train) // batch_size,
    validation_data=val_generator,
    verbose=1)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 55 steps, validate for 16 steps
Epoch 1/10
 8/55 [===>..........................] - ETA: 50s - loss: 11.8670

KeyboardInterrupt: 

In [7]:
prediction = model.predict_generator(test_generator)

In [8]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
RMSE = mean_squared_error(y_test, prediction, squared=False)
MAE = mean_absolute_error(y_test, prediction)
from scipy.stats import pearsonr
corr, t = pearsonr(y_test.flatten(), prediction.flatten())
RMSE,MAE,corr

(1.225973870894252, 0.9830283741696891, 0.2358935566958162)

In [3]:
#resnet50

import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.applications import MobileNetV2, ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import multi_gpu_model
from tensorflow.keras.models import load_model
import tensorflow.keras.backend as K

#basemodel = MobileNetV2(include_top=False, pooling='avg', weights='imagenet')
basemodel = ResNet50(include_top=False, pooling='avg', weights='imagenet')


X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.30, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=1/3, random_state=42)

from tensorflow.keras.preprocessing.image import ImageDataGenerator
test_val_datagen = ImageDataGenerator()
train_datagen = ImageDataGenerator(horizontal_flip=True,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2)

batch_size = 32
train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)
val_generator = test_val_datagen.flow(X_val, y_val, batch_size=batch_size)
test_generator = test_val_datagen.flow(X_test, y_test, batch_size=batch_size, shuffle=False)

model = Sequential()
model.add(basemodel)
model.add(Dense(1))
epochs = 10
lr=0.001
model.layers[0].trainable = False
model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=lr))
history1 = model.fit(
    train_generator,
    epochs=epochs,
    steps_per_epoch=len(y_train) // batch_size,
    validation_data=val_generator,
    verbose=1)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 24 steps, validate for 7 steps
Epoch 1/10
24/24 [==============================] - 52s 2s/step - loss: 15.0138 - val_loss: 3.9801
Epoch 2/10
24/24 [==============================] - 46s 2s/step - loss: 3.3589 - val_loss: 2.3063
Epoch 3/10
24/24 [==============================] - 47s 2s/step - loss: 2.2005 - val_loss: 2.0274
Epoch 4/10
24/24 [==============================] - 46s 2s/step - loss: 1.9750 - val_loss: 1.8914
Epoch 5/10
24/24 [==============================] - 46s 2s/step - loss: 1.8898 - val_loss: 1.6926
Epoch 6/10
24/24 [==============================] - 46s 2s/step - loss: 1.7254 - val_loss: 1.6168
Epoch 7/10
24/24 [==============================] - 47s 2s/step - loss: 1.6461 - val_loss: 1.6592
Epoch 8/10
24/24 [==============================] - 45s 2s/step - loss: 1.6082 - val_loss: 1.4290
Epoch 9/10
24/24 [==============================] - 45s 2s/step - loss: 1.5148 - val_loss: 1.4197
Epoch 10/10
24/24 [=======

In [14]:
MAE

1.0520501247124447

In [9]:
prediction

array([[5.4640684],
       [4.2668924],
       [4.475416 ],
       [4.310153 ],
       [5.0196815],
       [5.0651793],
       [5.463641 ],
       [1.9132888],
       [4.403923 ],
       [5.495001 ],
       [4.2097626],
       [4.2923627],
       [5.055629 ],
       [5.290922 ],
       [5.419596 ],
       [4.9922485],
       [4.9155684],
       [3.6595967],
       [4.6996546],
       [5.2225857],
       [2.6613078],
       [4.802147 ],
       [4.231185 ],
       [4.945115 ],
       [4.874496 ],
       [3.9225001],
       [4.348874 ],
       [5.464011 ],
       [5.4474635],
       [5.1206317],
       [4.5285664],
       [5.606208 ],
       [4.878786 ],
       [4.8046317],
       [5.2179136],
       [5.0540404],
       [5.8639345],
       [5.150322 ],
       [5.616313 ],
       [3.7388248],
       [3.9051042],
       [5.086461 ],
       [5.4133425],
       [4.3529644],
       [3.6266453],
       [4.6758866],
       [5.2126513],
       [5.048572 ],
       [4.82308  ],
       [4.4051533],


0.16833790278143046

In [16]:
corr, t

NameError: name 'corr' is not defined

In [4]:
model.save('/home/ubuntu/trained_models/resnet50_all_private_aligned_10epochs.h5')

2.3.0
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 55 steps, validate for 16 steps
Epoch 1/10
 1/55 [..............................] - ETA: 49s

InvalidArgumentError:  Incompatible shapes: [22,1] vs. [22,350,350,1]
	 [[node loss/dense_loss/SquaredDifference (defined at <ipython-input-4-58eb8cfbefce>:53) ]] [Op:__inference_distributed_function_12438]

Function call stack:
distributed_function


In [46]:
!pip3 keras

ERROR: unknown command "keras"


In [66]:
len(y_test)

255